'''In Excerise 3.02 Business Hypothesis Testing for Age versus Propensity for a Term Loan

Business Hypothesis Testing to Find Employment Status versus Propensity for Term Deposits
---




1. Formulate the hypothesis between employment status and the propensity for term deposits. Let the hypothesis be as follows: High paying employees prefer term deposits than other categories of employees.


In [1]:
#2. Install and import the necessary libraries such as pandas and altair.
import pandas as pd
import altair as alt

In [2]:
#Assign link to dataset
file_url = 'https://raw.githubusercontent.com/PacktWorkshops/The-Data-Science-Workshop/master/Chapter03/bank-full.csv'

# Loading the data using pandas
bank_data = pd.read_csv(file_url, sep=";")

In [4]:
#Check the data
bank_data.sample(10)

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
26698,43,self-employed,married,primary,no,5039,no,no,cellular,20,nov,227,1,-1,0,unknown,no
38655,32,blue-collar,single,secondary,no,574,yes,no,cellular,15,may,51,4,-1,0,unknown,no
33272,46,blue-collar,single,secondary,no,338,yes,no,cellular,20,apr,178,4,-1,0,unknown,no
5824,48,blue-collar,married,secondary,no,0,yes,no,unknown,26,may,105,1,-1,0,unknown,no
5047,34,management,single,tertiary,no,1717,yes,no,unknown,21,may,269,2,-1,0,unknown,no
14643,51,services,divorced,secondary,no,65,no,yes,cellular,15,jul,881,1,-1,0,unknown,yes
10711,36,admin.,single,secondary,no,16,no,no,unknown,16,jun,119,2,-1,0,unknown,no
5224,33,technician,single,tertiary,no,594,yes,yes,unknown,23,may,149,1,-1,0,unknown,no
20844,58,services,married,secondary,no,627,no,no,cellular,13,aug,110,4,-1,0,unknown,no
1890,26,entrepreneur,married,tertiary,no,79,yes,yes,unknown,9,may,346,3,-1,0,unknown,no


In [ ]:
#.shape function is used to find the overall shape of the dataset.
print(bank_data.shape)

(45211, 17)


In [ ]:
#find the summary of the numerical raw data as a table output using the .describe() 
bank_data.describe()

,age,balance,day,duration,campaign,pdays,previous
count,45211.000000,45211.000000,45211.000000,45211.000000,45211.000000,45211.000000,45211.000000
mean,40.936210,1362.272058,15.806419,258.163080,2.763841,40.197828,0.580323
std,10.618762,3044.765829,8.322476,257.527812,3.098021,100.128746,2.303441
min,18.000000,-8019.000000,1.000000,0.000000,1.000000,-1.000000,0.000000
25%,33.000000,72.000000,8.000000,103.000000,1.000000,-1.000000,0.000000
50%,39.000000,448.000000,16.000000,180.000000,2.000000,-1.000000,0.000000
75%,48.000000,1428.000000,21.000000,319.000000,3.000000,-1.000000,0.000000
max,95.000000,102127.000000,31.000000,4918.000000,63.000000,871.000000,275.000000


In [5]:
#count the number of records for each age group.
#Take DataFrame, bankData and then filter it for all cases where the term deposit is yes using the mask bankData['y'] == 'yes'. 
filter_mask = bank_data['y'] == 'yes'

#These cases are grouped through the groupby() method and then aggregated according to job through the agg() method. 
#Finally we need to use .reset_index() to get a well-structure DataFrame
bankSub1 = bank_data[filter_mask].groupby('job')['y'].agg(agegrp='count').reset_index()

bankSub1 

,job,agegrp
0,admin.,631
1,blue-collar,708
2,entrepreneur,123
3,housemaid,109
4,management,1301
5,retired,516
6,self-employed,187
7,services,369
8,student,269
9,technician,840


From the banking DataFrame, bankData, find the distribution of employment status using the .groupby(), .agg() and .reset_index() methods.

Group the data with respect to employment status using the .groupby() method and find the total count of propensities for each employment status using the .agg() method.

In [6]:
#Group the data per age using the groupby() method and find the total number of customers under each job group using the agg() method
# Getting another perspective
jobGrpTotal = bank_data.groupby('job')['y'].agg(jobGrpTotal='count').reset_index()

jobGrpTotal.sample(5)

,job,jobGrpTotal
4,management,9458
10,unemployed,1303
8,student,938
1,blue-collar,9732
11,unknown,288


In [7]:
#group the data by both job and propensity of purchase and find the total counts under each category of propensity, which are yes and no
# Getting all the details in one place
jobProp = bank_data.groupby(['job','y'])['y'].agg(jobCat='count').reset_index()

jobProp.sample(5)

,job,y,jobCat
10,retired,no,1748
15,services,yes,369
9,management,yes,1301
11,retired,yes,516
0,admin.,no,4540


In [10]:
#Merge both of these DataFrames based on the job variable using the pd.merge() function, 
#and then divide each category of propensity within each job group by the total customers 
#in the respective job group to get the proportion of customers. 

# Merging both the data frames
jobComb = pd.merge(jobProp, jobGrpTotal, left_on = ['job'], right_on=['job'])

jobComb['catProp'] = (jobComb.jobCat/jobComb.jobGrpTotal)*100

jobComb.sample(5)

,job,y,jobCat,jobGrpTotal,catProp
9,management,yes,1301,9458,13.755551
2,blue-collar,no,9024,9732,92.725031
20,unemployed,no,1101,1303,84.497314
3,blue-collar,yes,708,9732,7.274969
14,services,no,3785,4154,91.116996


In [9]:
# Visualising the relationship using altair
#Using columns from Datframe noted above for job, newly created column catProp, and y (yes/no)
alt.Chart(jobComb, title="Propensity of purchase by job").mark_bar().encode(alt.X('job',title=''), 
                                     alt.Y('catProp', title='Propensity Proportion'), color=alt.Color('y', legend=alt.Legend(title='')))

alt.Chart(...)

The chart from the solution and noted above are different because altair was used above as done in Exercise 3.02 and solution used dfply, ggplot, and matplotlib.

#Some visual exploration with plotly

In [11]:
!pip install plotly

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [19]:
# import additional libraries
from pandas import set_option
import plotly.express as px


In [20]:
set_option('display.width',150)
set_option('precision',3)

In [21]:
# Summary statistics of the numeric values
bank_data.describe()

,age,balance,day,duration,campaign,pdays,previous
count,45211.000,45211.000,45211.000,45211.000,45211.000,45211.000,45211.000
mean,40.936,1362.272,15.806,258.163,2.764,40.198,0.580
std,10.619,3044.766,8.322,257.528,3.098,100.129,2.303
min,18.000,-8019.000,1.000,0.000,1.000,-1.000,0.000
25%,33.000,72.000,8.000,103.000,1.000,-1.000,0.000
50%,39.000,448.000,16.000,180.000,2.000,-1.000,0.000
75%,48.000,1428.000,21.000,319.000,3.000,-1.000,0.000
max,95.000,102127.000,31.000,4918.000,63.000,871.000,275.000


In [22]:
# Summary statistics of the categorical values
bank_data.describe(include ='object')

,job,marital,education,default,housing,loan,contact,month,poutcome,y
count,45211,45211,45211,45211,45211,45211,45211,45211,45211,45211
unique,12,3,4,2,2,2,3,12,4,2
top,blue-collar,married,secondary,no,yes,no,cellular,may,unknown,no
freq,9732,27214,23202,44396,25130,37967,29285,13766,36959,39922


In [27]:
# plotting the bubble chart
fig = px.scatter(bank_data, x="job", y="balance")
 
# showing the plot
fig.show()

In [32]:
# plotting the violin chart
fig = px.violin(bank_data, x="job", y="duration")
 
# showing the plot
fig.show()

In [35]:
# plotting the figure - 3D Line Plot
fig = px.line_3d(bank_data, x="month", y="duration",
                 z="balance", color="month")
 
fig.show()